In [ ]:
# Downloaded the data using the following shell script and the header
    :

    getData()
    {
        CSV="pima-indians-diabetes.csv"
        wget -O ${CSV} https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data
        wget -O description https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.names
        HDR="pregnantNo,plasmaGlucose,bloodPressure,triceps,serumInsulin,bodyMass,diabetesPedigree,age,target"
        sed -i '1i\'"${HDR}" ${CSV}
    }

    getData

In [4]:
# All the imports required for HW3

import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import figure, output_file, show, vplot

In [5]:
# Read the csv file
df = pd.read_csv("pima-indians-diabetes.csv", header = 0).dropna()

to_predict = "target"

# since I did not have an hypothosis I chose all the fields
features = ['pregnantNo', 'plasmaGlucose', 'bloodPressure', 'triceps', 'serumInsulin', \
                'bodyMass', 'diabetesPedigree', 'age']

# feature Data
data = df[features]

# label data
label = df[to_predict]

In [6]:
# number of folds
folds = 5;

In [7]:
# tried couple of my own cross validate functions and finally took Justin's function.

def cross_validate_df(data, label, kfold, model):
    positions = data.index.values
    np.random.shuffle(positions)
    cv_score = 0

    for i in range(kfold):
        pos_var = len(data) / kfold

        test_slice = positions[i * pos_var: (i + 1) * pos_var]

        train_1 = positions[ : i * pos_var]
        train_2 = positions[ (i + 1) * pos_var : ]

        train_slice = np.concatenate([train_1, train_2])

        model.fit(data.loc[train_slice], label.loc[train_slice])
        k_score = model.score(data.loc[test_slice], label.loc[test_slice]) * 100;

        cv_score += k_score

    return cv_score / kfold

In [8]:
def NaiveBayes():
    # create a model for NaiveBayes
    model = MultinomialNB()

    # cross validate data
    print cross_validate_df(data, label, folds, model);

In [9]:
def knnNeighbor():
    # number of neighbors
    neighbors = 50

    nn = range(2, neighbors)

    #scoreArray
    scoreArray = []

    # for each neighbors
    for n in nn:
        # create a model with n neighbors
        model = KNeighborsClassifier(n)

        scoreArray.append(cross_validate_df(data, label, folds, model))

    plot = figure(title = "Neighbors vs Score")
    plot.line(nn, scoreArray)
    plot.xaxis.axis_label = "Neighbors"
    plot.yaxis.axis_label = "Score"

    output_file("diabetes.html", title = "Neighbors vs Score")

    p = vplot(plot)
    show(p)

In [10]:
# execute the KnnNeighbor
knnNeighbor()

In [11]:
# execute NaiveBayes
NaiveBayes()

60.1307189542


In [ ]:
# For the given Data set KnnNeighbor around 20 neighbors seems to give about 78% accuracy 
#     and NaiveBayes gives about 60%.